# Modelo preditivo para a tendência do IBOVESPA

O objetivo desse projeto é desenvolver um modelo de Machine Learning capaz de prever se o índice IBOVESPA vai fechar em alta ou baixa no dia seguinte, com acuracidade mínima de 75% em um conjunto de teste composto pelos últimos 30 dias de dados.

> "O Ibovespa é o principal indicador de desempenho das ações negociadas na B3 e reúne as empresas mais importantes do mercado de capitais brasileiro. Foi criado em 1968 e, ao longo desses 50 anos, consolidou-se como referência para investidores ao redor do mundo." [Referência](https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/ibovespa.htm)

## Importação das bibliotecas

In [ ]:
'''yfinance==0.2.65 pandas<2.0.0 numpy<2.0.0 scikit-learn==1.7.1 xgboost==3.0.2 matplotlib==3.10.3 ipython==9.4.0 jupyterlab==4.4.5 seaborn==0.13.2
pip install "numpy>=1.26,<2.0" "pandas_ta"'''

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
#import pandas_ta as ta

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

## Aquisição dos dados

Utilizaremos os dados históricos do índice IBOVESPA, disponíveis publicamente no site do [br.investing](https://br.investing.com/indices/bovespa-historical-data)

In [2]:
input_path = '../data/raw/dados_historicos_ibovespa_2008-2025.csv'

df = pd.read_csv(input_path, thousands='.', decimal=',', parse_dates=['Data'], date_format='%d.%m.%Y', index_col='Data')
df = df.rename_axis('ds').sort_index()
df.tail()

,Último,Abertura,Máxima,Mínima,Vol.,Var%
ds,,,,,,
2025-06-12,137800,137127,137931,136175,"7,12B","0,49%"
2025-06-13,137213,137800,137800,136586,"8,63B","-0,43%"
2025-06-16,139256,137212,139988,137212,"7,62B","1,49%"
2025-06-17,138840,139256,139497,138293,"8,38B","-0,30%"
2025-06-18,138717,138844,139161,138443,"8,32B","-0,09%"


In [3]:
# informações gerais do dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4315 entries, 2008-01-18 to 2025-06-18
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Último    4315 non-null   int64 
 1   Abertura  4315 non-null   int64 
 2   Máxima    4315 non-null   int64 
 3   Mínima    4315 non-null   int64 
 4   Vol.      4314 non-null   object
 5   Var%      4315 non-null   object
dtypes: int64(4), object(2)
memory usage: 236.0+ KB


## Tratamento dos dados

Vamos transformar os dados brutos em um formato adequado para o treinamento de um modelo de Machine Learning.

In [4]:
# renomeando as colunas para os nomes padrões utilizados no mercado financeiro
colunas = {
  'Último': 'close',              # fechamento da negociação diária
  'Abertura': 'open',             # início da negociação diária
  'Máxima': 'high',               # valor máximo do dia
  'Mínima': 'low',                # valor mínimo do dia
  'Vol.': 'volume',               # volume de negociação diária
  'Var%': 'daily_return'          # variação percentual diária
}

df.rename(columns=colunas, inplace=True)

In [5]:
# Data mínima, máxima e total de anos do DF levando em conta os anos bissextos
print(f"Os dados vão de {df.index.min().date()} até {df.index.max().date()}, o que dá aproximadamente {(df.index.max() - df.index.min()).days / 365.25:.0f} anos")

Os dados vão de 2008-01-18 até 2025-06-18, o que dá aproximadamente 17 anos


In [6]:
# conferindo se há valores duplicados
df.duplicated().sum()

np.int64(0)

In [7]:
# conferindo se há valores nulos
df.isna().sum().sort_values(ascending=False)

volume          1
close           0
open            0
high            0
low             0
daily_return    0
dtype: int64

In [8]:
df[df['volume'].isnull()]

,close,open,high,low,volume,daily_return
ds,,,,,,
2016-02-10,40377,40592,40592,39960,NaN,"-0,53%"


In [9]:
def converter_volume(vol: str | float) -> float:
    """
    Converte uma string de volume com sufixos (K, M, B) para um número float.
    
    Parâmetro:
        vol (string | float): o valor a ser convertido (ex: '8,3M'). Pode ser uma string ou um np.nan (que é float).
        
    Retorna:
        float: o valor convertido ou np.nan caso não haja um valor.
    """
    if not isinstance(vol, str):
        return vol

    multiplicadores = {'K': 1e3, 'M': 1e6, 'B': 1e9}
    vol = vol.upper().replace(',', '.').strip()
    sufixo = vol[-1]

    if sufixo in multiplicadores:
        return float(vol[:-1]) * multiplicadores[sufixo]
    else:
        return float(vol)

df['volume'] = df['volume'].apply(converter_volume)

In [10]:
# substituir o volume nulo pela média do volume anterior e posterior daquela data
df['volume'] = df['volume'].interpolate()

In [11]:
# ajustando a coluna variação percentual diária, que contém o pct_change() do fechamento
df['daily_return'] = df['daily_return'].str.replace('%', '').str.replace(',', '.')
df['daily_return'] = round(df['daily_return'].astype(float) / 100, 4)
df.head()

,close,open,high,low,volume,daily_return
ds,,,,,,
2008-01-18,57506,57039,58291,56241,5810000.0,0.0082
2008-01-21,53709,57503,57503,53487,3570000.0,-0.0660
2008-01-22,56097,53705,56541,53610,3650000.0,0.0445
2008-01-23,54235,56098,56098,53011,3720000.0,-0.0332
2008-01-24,57463,54242,57675,54242,3800000.0,0.0595


In [12]:
# conferindo formato dos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4315 entries, 2008-01-18 to 2025-06-18
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   close         4315 non-null   int64  
 1   open          4315 non-null   int64  
 2   high          4315 non-null   int64  
 3   low           4315 non-null   int64  
 4   volume        4315 non-null   float64
 5   daily_return  4315 non-null   float64
dtypes: float64(2), int64(4)
memory usage: 236.0 KB


## Engenharia de atributos

### Baseados em dados internos

Agora, vamos criar **variáveis preditoras** através de uma análise técnica do mercado financeiro, ao invés de uma análise fundamentalista. Em vez de dar ao modelo os dados brutos de abertura, máxima e mínima, vamos criar features para extrair a informação das relações entre eles.

Para as janelas de tempo, vamos utilizar as convenções do mercado financeiro para representar uma semana (5 dias), um mês (21 dias) e um trimestre (63 dias) de transações.

In [ ]:
# indicadores de momentum

# retorno percentual de 1 a 5 dias (tendência de curto prazo)
for lag in range(1, 7):
    df[f'return_lag_{lag}'] = df['daily_return'].shift(lag)

# retorno percentual acumulado de uma semana, um mês e um trimestre
for period in [5, 21, 63]:
    df[f'momentum_{period}'] = df['close'].pct_change(period)

In [14]:
df

,close,open,high,low,volume,daily_return,return_lag_1,return_lag_2,return_lag_3,return_lag_4,return_lag_5,momentum_5,momentum_21,momentum_63
ds,,,,,,,,,,,,,,
2008-01-18,57506,57039,58291,56241,5.810000e+06,0.0082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-21,53709,57503,57503,53487,3.570000e+06,-0.0660,0.0082,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-22,56097,53705,56541,53610,3.650000e+06,0.0445,-0.0660,0.0082,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-23,54235,56098,56098,53011,3.720000e+06,-0.0332,0.0445,-0.0660,0.0082,NaN,NaN,NaN,NaN,NaN
2008-01-24,57463,54242,57675,54242,3.800000e+06,0.0595,-0.0332,0.0445,-0.0660,0.0082,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-12,137800,137127,137931,136175,7.120000e+09,0.0049,0.0051,0.0054,-0.0030,-0.0010,-0.0056,0.011480,-0.004501,0.112510
2025-06-13,137213,137800,137800,136586,8.630000e+09,-0.0043,0.0049,0.0051,0.0054,-0.0030,-0.0010,0.008163,-0.015222,0.092138
2025-06-16,139256,137212,139988,137212,7.620000e+09,0.0149,-0.0043,0.0049,0.0051,0.0054,-0.0030,0.026212,0.000496,0.079864


A média móvel é uma técnica que suaviza as flutuações de curto prazo e destaca tendências de longo prazo. Enquanto a média móvel simples calcula a média aritmética em um período predeterminado, a média móvel exponencial dá mais peso aos valores mais recentes, tornando-a mais sensível a novas informações.

Quando uma média móvel de curto prazo cruza acima de uma média móvel de longo prazo, pode sugerir uma mudança de tendência de decrescimento para crescimento.

In [15]:
# 2. Seleção de Features (X)
# Adicionar lag do 'close' como feature

df['close_lag1'] = df['close'].shift(1)
df['close_lag3'] = df['close'].shift(3)
df['close_lag5'] = df['close'].shift(5)
df['close_lag7'] = df['close'].shift(7)

In [16]:
df

,close,open,high,low,volume,daily_return,return_lag_1,return_lag_2,return_lag_3,return_lag_4,return_lag_5,momentum_5,momentum_21,momentum_63,close_lag1,close_lag3,close_lag5,close_lag7
ds,,,,,,,,,,,,,,,,,,
2008-01-18,57506,57039,58291,56241,5.810000e+06,0.0082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-21,53709,57503,57503,53487,3.570000e+06,-0.0660,0.0082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57506.0,NaN,NaN,NaN
2008-01-22,56097,53705,56541,53610,3.650000e+06,0.0445,-0.0660,0.0082,NaN,NaN,NaN,NaN,NaN,NaN,53709.0,NaN,NaN,NaN
2008-01-23,54235,56098,56098,53011,3.720000e+06,-0.0332,0.0445,-0.0660,0.0082,NaN,NaN,NaN,NaN,NaN,56097.0,57506.0,NaN,NaN
2008-01-24,57463,54242,57675,54242,3.800000e+06,0.0595,-0.0332,0.0445,-0.0660,0.0082,NaN,NaN,NaN,NaN,54235.0,53709.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-12,137800,137127,137931,136175,7.120000e+09,0.0049,0.0051,0.0054,-0.0030,-0.0010,-0.0056,0.011480,-0.004501,0.112510,137128.0,135699.0,136236.0,137546.0
2025-06-13,137213,137800,137800,136586,8.630000e+09,-0.0043,0.0049,0.0051,0.0054,-0.0030,-0.0010,0.008163,-0.015222,0.092138,137800.0,136436.0,136102.0,137002.0
2025-06-16,139256,137212,139988,137212,7.620000e+09,0.0149,-0.0043,0.0049,0.0051,0.0054,-0.0030,0.026212,0.000496,0.079864,137213.0,137128.0,135699.0,136236.0


In [16]:
# indicadores de tendência

# média móvel simples de um mês (tendência de médio prazo)
df['sma_21'] = ta.sma(df['close'], length=21)

# média móvel exponencial de um trimestre (tendência de longo prazo)
df['ema_50'] = ta.ema(df['close'], length=50)

Para a média móvel exponencial, é uma convenção utilizar 50 dias ou 200 dias para tendências de longo prazo. [Referência](https://www.investopedia.com/terms/e/ema.asp#citation-4)

In [17]:
# indicadores de osciladores

# índice de força relativa (IFR) varia de 0 a 100
df['rsi_14'] = ta.rsi(df['close'], length=14)

Por padrão, utiliza-se um período de 14 dias para o IFR. [Referência](https://blog.quantinsti.com/rsi-indicator/)

In [18]:
# indicadores de volatilidade

# average true range
df['atr_14'] = ta.atr(df['high'], df['low'], df['close'], length=14)

Por padrão, utiliza-se um período de 14 dias para o ATR. [Referência](https://www.investopedia.com/terms/a/atr.asp)

Por padrão, utiliza-se um período de 20 dias (um mês) para e 2 desvios-padrão para as bandas para conter cerca de 95% dos dados. [Referência](https://www.infomoney.com.br/guias/bandas-bollinger/)

In [19]:
# indicadores de volume

# on-balance volume 
df['obv'] = ta.obv(df['close'], df['volume'])

Decidimos não usar as features de volume, uma vez que são dados com muito ruído e que vão de uma escala de mil até bilhão. (vide notebook EDA)

In [20]:
# indicadores de data

# dia da semana (segunda=0 a sexta=4)
df['day_of_week'] = df.index.dayofweek

# dia do mês (1 a 30 ou 31)
df['day_of_month'] = df.index.day

# mês (janeiro=1 a dezembro=12)
df['month'] = df.index.month

# ano (pode não ser bom para previsões de curto prazo)
df['year'] = df.index.year

### Baseados em dados externos

> "O desempenho do mercado de ações ao redor do mundo afeta diretamente o comportamento da bolsa brasileira" [Referência](https://investnews.com.br/guias/bolsas-internacionais-quais-sao-como-afetam-ibovespa/) e [Referência](https://einvestidor.estadao.com.br/mercado/como-bolsas-internacionais-afetam-b3-ibovespa/)

* S&P 500 (`^GSPC`): Um dos principais índices da NYSE, o S&P 500 inclui 500 empresas líderes e representa aproximadamente 80% da capitalização de mercado disponível.
* Índice SSE Composite (`000001.SS`): O principal índice do Shanghai Stock Exchange, o SSEC, abarca ações de grandes empresas da China, como a Alibaba (BABA34), o Bank of China e a China Petrol.
* Euronext 100 (`^N100`): O índice Euronext 100, engloba companhia de alguns países da União Europeia, e tem ações de empresas como L’Oréal, Louis Vuitton e Renault.

* Petróleo tipo Brent (`BZ=F`): O valor do barril da commodity interfere no preço das ações. No caso da Petrobras, o barril do tipo Brent é a principal referência. [Referência](https://borainvestir.b3.com.br/noticias/empresas/quais-os-principais-fatores-que-afetam-as-acoes-da-petrobras/)

In [ ]:
'''tickers = ['^GSPC', '000001.SS', '^N100', 'BZ=F']
df_global = yf.download(tickers, start='2010-01-01', end='2025-06-19', auto_adjust=True, multi_level_index=False)
df_global.head()'''

NameError: name 'yf' is not defined

In [ ]:
'''# selecionar apenas a coluna de fechamento
df_global = df_global['Close']

# lidar com dados ausentes (feriados de outros mercados)
df_global.ffill(inplace=True)'''

In [ ]:
'''# calcular retorno diário com defasagem (do dia anterior)
external_features = pd.DataFrame(index=df_global.index)

external_features ['eua_return_lag1'] = df_global['^GSPC'].pct_change().shift(1)
external_features ['china_return_lag1'] = df_global['000001.SS'].pct_change().shift(1)
external_features ['europe_return_lag1'] = df_global['^N100'].pct_change().shift(1)
external_features ['oil_return_lag1'] = df_global['BZ=F'].pct_change().shift(1)

external_features.head()'''

,eua_return_lag1,china_return_lag1,europe_return_lag1,oil_return_lag1
Date,,,,
2010-01-04,NaN,NaN,NaN,NaN
2010-01-05,NaN,NaN,NaN,NaN
2010-01-06,0.003116,0.011844,0.000832,0.005866
2010-01-07,0.000546,-0.008520,0.001004,0.016131
2010-01-08,0.004001,-0.018880,-0.000730,-0.004640


In [ ]:
'''# juntar com o dataframe do ibovespa
df_merged = df.merge(external_features, left_index=True, right_index=True, how='left')'''

## Exportação dos dados

In [22]:
# selecionando apenas 10 anos de dados
#df_10years = df_merged.loc[df.index >= '2015-06-17']
df_10years = df[df.index >= '2015-06-17']
df_10years.head()

,close,open,high,low,volume,daily_return,return_lag_1,return_lag_2,return_lag_3,return_lag_4,...,close_lag7,sma_21,ema_50,rsi_14,atr_14,obv,day_of_week,day_of_month,month,year
ds,,,,,,,,,,,,,,,,,,,,,
2015-06-17,53249,53698,53755,52965,3090000.0,-0.0084,0.0106,-0.0039,-0.0064,-0.0035,...,52810.0,53785.285714,53919.575856,44.356672,889.081392,78500090.0,2,17,6,2015
2015-06-18,54239,53251,54352,53214,2750000.0,0.0186,-0.0084,0.0106,-0.0039,-0.0064,...,52816.0,53725.285714,53932.102293,51.525407,906.861293,81250090.0,3,18,6,2015
2015-06-19,53749,54236,54236,53479,2950000.0,-0.0090,0.0186,-0.0084,0.0106,-0.0039,...,53876.0,53670.428571,53924.921811,48.214460,896.371201,78300090.0,4,19,6,2015
2015-06-22,53864,53750,54342,53655,2430000.0,0.0021,-0.0090,0.0186,-0.0084,0.0106,...,53689.0,53611.000000,53922.532720,49.042076,881.416115,80730090.0,0,22,6,2015
2015-06-23,53772,53865,54361,53772,2710000.0,-0.0017,0.0021,-0.0090,0.0186,-0.0084,...,53348.0,53582.190476,53916.629476,48.375998,860.529250,78020090.0,1,23,6,2015


In [23]:
df_10years.isnull().sum()

close           0
open            0
high            0
low             0
volume          0
daily_return    0
return_lag_1    0
return_lag_2    0
return_lag_3    0
return_lag_4    0
return_lag_5    0
momentum_5      0
momentum_21     0
momentum_63     0
close_lag1      0
close_lag3      0
close_lag5      0
close_lag7      0
sma_21          0
ema_50          0
rsi_14          0
atr_14          0
obv             0
day_of_week     0
day_of_month    0
month           0
year            0
dtype: int64

In [24]:
# Salvar o CSV já limpo e processado com a data no índice
output_path = '../data/processed/dados_historicos_ibovespa_2015-2025_processed.csv'

df_10years.to_csv(output_path, index=True)